In [1]:
import numpy as np
import requests
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
import time 
import pandas as pd
import json
import requests
from pyquery import PyQuery

In [2]:
def get_links(page_health, page_general, page_politics, page_business, page_world):
    # Extraction of all links till june 2020
    service = Service(ChromeDriverManager().install())


    #driver = webdriver.Chrome(service=service)
    browser = webdriver.Chrome(service=service)
    urls = []
    counter = 2

    articles_link = ["Links"]
    topic = ["topic"]
    # since the tops category has 7 pages, where link to each follows a specific pattern identified above, we can create links to them as following:
    urls.append('https://digiteye.in/category/health/')
    topic.append('health')
    for i in range(2, 26):
        urls.append( 'https://digiteye.in/category/health/page/' + str(counter) + '/')
        topic.append('health')
        counter += 1

    counter = 2
    urls.append('https://digiteye.in/category/general/')
    topic.append('general')
    for i in range(2, 94):
        urls.append( 'https://digiteye.in/category/general/page/' + str(counter) + '/')
        topic.append('general')
        counter += 1

    counter = 2
    urls.append('https://digiteye.in/category/politics/')
    topic.append('politics')
    for i in range(2, 32):
        urls.append( 'https://digiteye.in/category/politics/page/' + str(counter) + '/')
        topic.append('politics')
        counter += 1

    counter = 2
    urls.append('https://digiteye.in/category/business/')
    topic.append('business')
    for i in range(2, 10):
        urls.append( 'https://digiteye.in/category/business/page/' + str(counter) + '/')
        topic.append('business')
        counter += 1

    counter = 2
    urls.append('https://digiteye.in/category/world/')
    topic.append('world')
    for i in range(2, 23):
        urls.append( 'https://digiteye.in/category/world/page/' + str(counter) + '/')
        topic.append('world')
        counter += 1

    # extracting links for products in each page
    for url in urls:
        # open the url
        browser.get(url)
        # purposeful wait time to allow the website to get fully loaded
        time.sleep(4)
        # get page content
        content = browser.page_source
        soup = BeautifulSoup(content, "lxml")
        art_links = []
        # extract all the anchor i.e., <a> elements with “thumb-link” class from the page
        data_links = soup.find_all("a", {"class": "more-link"})
        # from each <a> element, extract the URL
        for i in data_links:
            art_links.append(i['href'])

        articles_link.extend(art_links)
        # purposeful wait time to avoid sending requests in quick succession
        time.sleep(10)
    browser.quit()
    np.savetxt("digiteye_english_links_june2020.csv",
        articles_link,
        delimiter =", ",
        fmt ='% s')
    return

In [3]:
def restore_unicode(mangled):
    return mangled.encode('latin1','ignore').decode('utf8', 'replace')
def get_article_content(pq):
    
    content = {
        "text": [],
        "video": [],
        "image": [],
        "tweet": [],
        "facebook": [],
        "instagram": [],
    }

    ## text content
    content['text'] = restore_unicode(pq('div.entry').text())

    ## images
    images = pq.find('figure>img')
    images += pq.find('.image-and-caption-wrapper>img')
    images += pq.find('.single-post-thumb>img')
    images = list(dict.fromkeys(images))

    for i in images:
        if 'src' in i.attrib:
            content["image"].append(i.attrib["src"])

    ## video embed
    video_embed = pq.find("video>source")
    for v in video_embed:
        content["video"].append(v.attrib["src"])
    
    video_yt = pq.find('iframe')  # video youtube
    for v in video_yt:
        if 'lazy' in v.attrib.get('class', ''):
            continue
        content["video"].append(v.attrib["src"])
   
    fb = pq.find('.wp-block-embed-facebook>.fb-video')   # video fb
    for f in fb:
        content["facebook"].append(f.attrib["data-href"])
    return content

In [4]:
def get_content(dt):
    for url in dt["Links"]:
        #url = "https://digiteye.in/sadhguru-dancing-when-coronavirus-is-around-fact-check/"

        headers = {
            'User-Agent': 'Your User-Agent String Here',
        }

        response = requests.get(url, headers=headers)
        # Check if the request was successful
        if response.status_code == 200:
            # Parse the HTML content of the page using BeautifulSoup
            soup = BeautifulSoup(response.text, 'html.parser')
            meta_tags = soup.find_all('meta')
            meta_features = {}
            for tag in meta_tags:
                if tag.get('property'):
                    meta_features[tag['property']] = tag.get('content')

            # Published_date
            json_data = json.loads(soup.find(type="application/ld+json").string)
            Published_date.append(json_data['@graph'][0]['datePublished'])
            # title
            title.append(json_data['@graph'][0]['headline'])

            # thumbnail
            #thumbnail.append(json_data['@graph'][3]['itemListElement'][1]['item'][36:-1])
            topic.append(json_data['@graph'][0]['articleSection'])

            # Content
            data = soup.find_all('div', {'class':"entry"})
            if data:
                extracted_content = []
                for element in data:
                    for el in element.find_all(['p', 'blockquote']):
                        extracted_content.append(el.get_text())

                full_content = '\n'.join(extracted_content)

            Content.append(full_content)

            link = []
            liss = data[0].find_all('a')
            for i in liss:
                link.append(i['href'])
            links_in_text.append(link)

            html_text = response.content
            pq = PyQuery(html_text)
            article_content = get_article_content(pq)
            thumbnail.append(article_content['image'])

            Linnnks.append(url)
    # save to csv file
    df = pd.DataFrame({'link':Linnnks, 'title': title, 'publish_date': Published_date, 'content': Content, 'Links_in_text':links_in_text, 'thumbnail': thumbnail,'topic':topic})
    df.to_csv('digiteye_kanada_june2020.csv')
    return

In [8]:
if __name__ == "__main__":
    title = []
    Published_date = []
    Content = []
    video = []
    thumbnail = []
    links_in_text = []
    topic = []
    Linnnks = []
    get_links(26, 94, 32, 10, 23)
    dataframe = pd.read_csv("digiteye_kanada_links_june2020.csv")
    get_content(dataframe)